## Prepare and read data

In [1]:
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
import lightgbm as lgb

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_squared_error

In [2]:
pd.set_option("max_columns", 300)
pd.set_option("max_rows", 500)

In [3]:
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")

## Feature Engineering

In [4]:
def get_multi_cat_cols(train_df):
    tmp = train_df.iloc[:1000]
    multi_cols = []
    for c in train_df.columns:
        sep_num = tmp[c].astype(str).fillna("").str.contains(";").sum()
        if sep_num > 10:
            multi_cols.append(c)
    return multi_cols

In [5]:
original_cols = train_df.columns.tolist()

In [6]:
multi_cat_cols = get_multi_cat_cols(train_df)

In [7]:
for c in tqdm(multi_cat_cols):
    binarizer = MultiLabelBinarizer()
    train_multi_srs = train_df[c].map(lambda x: x.split(";") if x is not np.nan else [])
    test_multi_srs = test_df[c].map(lambda x: x.split(";") if x is not np.nan else [])
    train_arr = binarizer.fit_transform(train_multi_srs)
    test_arr = binarizer.transform(test_multi_srs)
    feat_cols = [f"{c}_{val}" for val in binarizer.classes_]
    train_feat_df = pd.DataFrame(train_arr, columns=feat_cols)
    test_feat_df = pd.DataFrame(test_arr, columns=feat_cols)
    train_df = pd.concat([train_df, train_feat_df], axis=1)
    test_df = pd.concat([test_df, test_feat_df], axis=1)

In [8]:
other_cols = [
    c for c in list(np.setdiff1d(original_cols, multi_cat_cols)) if c not in ["Salary", "No"]
]

In [9]:
for c in tqdm(other_cols):
    if train_df[c].dtype.name == "float":
        continue
    else:
        train_df[c], uniques = pd.factorize(train_df[c])
        test_df[c] = uniques.get_indexer(test_df[c])

In [10]:
train_df.shape, test_df.shape

((33857, 444), (11259, 443))

In [11]:
train_df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in train_df.columns]
test_df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in test_df.columns]

In [12]:
use_cols = [c for c in train_df.columns if c not in multi_cat_cols + ["Salary", "No"]]

In [13]:
folds = KFold(n_splits=5, random_state=2020, shuffle=True)
fold_idx_list = [(trn_idx, val_idx) for trn_idx, val_idx in folds.split(train_df, train_df["Salary"])]

### Target Encoding

In [14]:
for c in tqdm(use_cols):
    if train_df[c].nunique() < 5:
        continue
    new_col = "te_" + c
    new_col_std = "te_std_" + c
    train_df[new_col] = 0
    test_df[new_col] = 0
    train_df[new_col_std] = 0
    test_df[new_col_std] = 0
    for trn_idx, val_idx in fold_idx_list:
        mean_val = train_df.loc[trn_idx].groupby(c)["Salary"].mean()
        std_val = train_df.loc[trn_idx].groupby(c)["Salary"].std()
        train_df.loc[val_idx, new_col] = train_df.loc[val_idx, c].map(mean_val)
        test_df.loc[:, new_col] += test_df.loc[:, c].map(mean_val) / len(fold_idx_list)
        train_df.loc[val_idx, new_col_std] = train_df.loc[val_idx, c].map(std_val)
        test_df.loc[:, new_col_std] += test_df.loc[:, c].map(std_val) / len(fold_idx_list)

In [15]:
for c in tqdm(use_cols):
    if train_df[c].nunique() < 10:
        continue
    new_col = "te_millitary_" + c
    train_df[new_col] = 0
    test_df[new_col] = 0
    train_df["tmp"] = train_df["MilitaryUS"].astype(str) + train_df[c].fillna("").astype(str)
    test_df["tmp"] = test_df["MilitaryUS"].astype(str) + test_df[c].fillna("").astype(str)
    for trn_idx, val_idx in fold_idx_list:
        mean_val = train_df.loc[trn_idx].groupby("tmp")["Salary"].mean()
        train_df.loc[val_idx, new_col] = train_df.loc[val_idx, "tmp"].map(mean_val)
        test_df.loc[:, new_col] += test_df.loc[:, "tmp"].map(mean_val) / len(fold_idx_list)
    del train_df["tmp"], test_df["tmp"]

In [16]:
use_cols = [c for c in train_df.columns if c not in multi_cat_cols + ["Salary", "No"]]

In [17]:
len(use_cols)

630

## Training

In [18]:
lgb_params = {
            'objective': 'regression',
            "metric": "rmse",
            "verbosity": -1,
            "boosting": "gbdt",
            'learning_rate': 0.05,
            'num_leaves': 128,
            'min_data_in_leaf': 30, 
            'max_depth': 6,
            "bagging_freq": 10,
            "bagging_fraction": 0.4,
            "lambda_l1": 0.5,
            "lambda_l2": 0.5,
            "feature_fraction": 0.7,
            "seed": 2020,
            "num_threads": -1,
            "max_bins": 30
}

In [19]:
importances = pd.DataFrame()
oof_preds = np.zeros(len(train_df))
models = []

for fold_i, (trn_idx, val_idx) in enumerate(fold_idx_list):
    print(f"Fold {fold_i+1}")
    train_dataset = lgb.Dataset(
        train_df.loc[trn_idx, use_cols],
        label = train_df.loc[trn_idx, "Salary"]
    )
    valid_dataset = lgb.Dataset(
        train_df.loc[val_idx, use_cols],
        label = train_df.loc[val_idx, "Salary"]
    )
    model = lgb.train(
                lgb_params,
                train_dataset,
                3000,
                valid_sets = [train_dataset, valid_dataset],
                verbose_eval=100,
                early_stopping_rounds = 100,
                #feval = eval_f1,
                #callbacks = [log_callback],
    )
    imp_df = pd.DataFrame()
    imp_df['feature'] = use_cols
    imp_df['gain'] = model.feature_importance(importance_type="gain")
    importances = pd.concat([importances, imp_df], axis=0, sort=False)
    
    oof_preds[val_idx] = model.predict(train_df.loc[val_idx, use_cols])
    models.append(model)

Fold 1
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 19514.7	valid_1's rmse: 21628.4
[200]	training's rmse: 17767.7	valid_1's rmse: 21280
[300]	training's rmse: 16626.4	valid_1's rmse: 21215
[400]	training's rmse: 15686.4	valid_1's rmse: 21264.1
Early stopping, best iteration is:
[310]	training's rmse: 16527.8	valid_1's rmse: 21192.6
Fold 2
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 19491.5	valid_1's rmse: 21617
[200]	training's rmse: 17826.9	valid_1's rmse: 21133.3
[300]	training's rmse: 16616.4	valid_1's rmse: 20995.5
[400]	training's rmse: 15645	valid_1's rmse: 20908.4
[500]	training's rmse: 14920.1	valid_1's rmse: 20881.8
[600]	training's rmse: 14236.5	valid_1's rmse: 20892.8
Early stopping, best iteration is:
[536]	training's rmse: 14672.5	valid_1's rmse: 20866
Fold 3
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 19472	valid_1's rmse: 21581
[200]	training's rmse:

In [20]:
oof_score = np.sqrt(mean_squared_error(train_df["Salary"], oof_preds))
oof_score

20996.52881418939

In [21]:
importances.groupby("feature")["gain"].mean().sort_values(ascending=False).reset_index()

,feature,gain
0,te_millitary_YearsCodingProf,5.500096e+13
1,te_Country,5.179469e+13
2,te_YearsCodingProf,9.685746e+12
3,te_millitary_Country,9.058884e+12
4,te_Employment,3.935681e+12
...,...,...
625,IDE_Zend,0.000000e+00
626,PlatformWorkedWith_Google_Home,0.000000e+00
627,LanguageDesireNextYear_Cobol,0.000000e+00
628,LanguageWorkedWith_Hack,0.000000e+00


In [22]:
test_pred = np.zeros(len(test_df))

for model in models:
    test_pred += model.predict(test_df[use_cols]) / len(models)

In [23]:
test_pred

array([ 70121.72316298,  93860.65966288,  84339.25290921, ...,
        93316.78197578,  81915.1281221 , 112460.3037865 ])

In [24]:
sub_df = pd.read_csv("../input/submit.csv")

In [25]:
sub_df["Salary"] = test_pred

In [26]:
# sub_df.to_csv("../predict/004_baseline_te_20975.csv", index=False)